# 1. Import Libraries

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import copy
import time

# 2. Load Data

In [17]:
# Define transformations specific to EfficientNetV2
transform = transforms.Compose([
    transforms.Resize((300, 300)),  # Specific size for EfficientNetV2
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dir = 'output_dataset/train'
val_dir = 'output_dataset/val'
test_dir = 'output_dataset/test'

train_dataset = ImageFolder(root=train_dir, transform=transform)
val_dataset = ImageFolder(root=val_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Dataset sizes
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

# Data loaders dictionary
dataloaders = {
    'train': train_loader,
    'val': val_loader
}


# 3. Define the Model

In [18]:
# Load pre-trained EfficientNetV2 model
model = models.efficientnet_v2_s(pretrained=True)

# Modify the final fully connected layer for your specific task
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# 4. Fine-tune the Model

In [19]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training function
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Initialize history dictionary to store metrics
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc * 100:.4f}')

            # Store in history
            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())  # Convert tensor to float

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc * 100:.4f}')

    model.load_state_dict(best_model_wts)
    return model, history

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model, history = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=15)



Epoch 1/15
train Loss: 1.1419 Acc: 51.1905
val Loss: 6.0453 Acc: 17.7778

Epoch 2/15
train Loss: 1.0303 Acc: 58.8095
val Loss: 1.2880 Acc: 41.1111

Epoch 3/15
train Loss: 0.7727 Acc: 68.0952
val Loss: 1.2831 Acc: 52.2222

Epoch 4/15
train Loss: 0.6501 Acc: 75.9524
val Loss: 1.0251 Acc: 70.0000

Epoch 5/15
train Loss: 0.5831 Acc: 81.6667
val Loss: 1.2440 Acc: 51.1111

Epoch 6/15
train Loss: 0.5071 Acc: 81.6667
val Loss: 1.4216 Acc: 63.3333

Epoch 7/15
train Loss: 0.5471 Acc: 81.6667
val Loss: 1.0551 Acc: 64.4444

Epoch 8/15
train Loss: 0.2620 Acc: 92.3810
val Loss: 0.9872 Acc: 64.4444

Epoch 9/15
train Loss: 0.2215 Acc: 93.8095
val Loss: 0.8717 Acc: 67.7778

Epoch 10/15
train Loss: 0.1329 Acc: 95.9524
val Loss: 0.9541 Acc: 68.8889

Epoch 11/15
train Loss: 0.1013 Acc: 96.9048
val Loss: 0.9887 Acc: 67.7778

Epoch 12/15
train Loss: 0.1245 Acc: 96.1905
val Loss: 0.9548 Acc: 70.0000

Epoch 13/15
train Loss: 0.0677 Acc: 98.3333
val Loss: 1.0229 Acc: 67.7778

Epoch 14/15
train Loss: 0.0528 Ac

# 5. Save the Model

In [20]:
# Save the trained model
torch.save(model.state_dict(), 'efficientnetv2_baseline.pth')

# 6. Prune the Model

### 6.1 Define baseline model

In [21]:
import torch
import torch.nn as nn
import torchvision.models as models

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained EfficientNetV2 model
model = models.efficientnet_v2_s(pretrained=True)

num_classes = 4 
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Load your own saved trained weights
model.load_state_dict(torch.load('efficientnetv2_baseline.pth'))

# Move model to device
model = model.to(device)

### 6.2 Prunning configuration with 30% Weights pruned

In [22]:
import torch.nn.utils.prune as prune

def prune_and_remove(model, amount=0.3):
    """
    Prunes 30% of weights in all Conv2d and Linear layers, then makes it permanent.
    """
    for name, module in model.named_modules():
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Apply unstructured L1 pruning
            prune.l1_unstructured(module, name='weight', amount=amount)
            # Remove the pruning mask and make it permanent
            prune.remove(module, 'weight')
    return model

pruned_model = prune_and_remove(model, amount=0.3)

### 6.3 Train pruned model and save it

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pruned_model.parameters(), lr=0.0001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Train again
pruned_model, pruned_history = train_model(pruned_model, criterion, optimizer, exp_lr_scheduler, num_epochs=15)

# Save pruned model
torch.save(pruned_model.state_dict(), 'efficientnetv2_pruned.pth')


Epoch 1/15
train Loss: 0.3949 Acc: 86.1905
val Loss: 0.8649 Acc: 70.0000

Epoch 2/15
train Loss: 0.2314 Acc: 91.6667
val Loss: 1.1133 Acc: 68.8889

Epoch 3/15
train Loss: 0.1626 Acc: 94.7619
val Loss: 1.0957 Acc: 70.0000

Epoch 4/15
train Loss: 0.1004 Acc: 97.6190
val Loss: 1.1827 Acc: 68.8889

Epoch 5/15
train Loss: 0.0685 Acc: 98.0952
val Loss: 1.1387 Acc: 71.1111

Epoch 6/15
train Loss: 0.0439 Acc: 99.2857
val Loss: 1.2786 Acc: 71.1111

Epoch 7/15
train Loss: 0.0424 Acc: 99.0476
val Loss: 1.2527 Acc: 71.1111

Epoch 8/15
train Loss: 0.0340 Acc: 99.2857
val Loss: 1.1881 Acc: 70.0000

Epoch 9/15
train Loss: 0.0411 Acc: 98.8095
val Loss: 1.2284 Acc: 70.0000

Epoch 10/15
train Loss: 0.0541 Acc: 98.3333
val Loss: 1.1977 Acc: 70.0000

Epoch 11/15
train Loss: 0.0699 Acc: 97.8571
val Loss: 1.3282 Acc: 71.1111

Epoch 12/15
train Loss: 0.0429 Acc: 99.2857
val Loss: 1.3577 Acc: 71.1111

Epoch 13/15
train Loss: 0.0275 Acc: 100.0000
val Loss: 1.2364 Acc: 70.0000

Epoch 14/15
train Loss: 0.0308 A

# 7. Evaluate the Model

In [ ]:
# Define evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Evaluate baseline model
model.load_state_dict(torch.load('efficientnetv2_baseline.pth'))
baseline_accuracy = evaluate_model(model, test_loader)
print(f'Baseline Model Accuracy: {baseline_accuracy:.2f}%')

# Evaluate pruned model
model.load_state_dict(torch.load('efficientnetv2_pruned.pth'))
pruned_accuracy = evaluate_model(model, test_loader)
print(f'Pruned Model Accuracy: {pruned_accuracy:.2f}%')

Baseline Model Accuracy: 89.13%
Pruned Model Accuracy: 89.83%
